In [1]:
%reload_ext autoreload
%autoreload 2

## Setup

In [2]:
from math import isclose
from ipyexperiments import IPyExperiments
import ipyexperiments
import re, numpy as np

In [3]:
def consume_cpu(n): return np.ones((n, n))

In [4]:
def check_defined(var_list, local_list): 
    for v in var_list: assert v in local_list, f"var {v} should exist in locals()"
        
def check_undefined(var_list, local_list):
    for v in var_list: assert v not in local_list, f"var {v} should not exist in locals()"

# --------------------------------------------------------------------- #
# the following functions work with the captured output
# output is captured by `%%capture output` from a cell before
def get_consumed_size(output):
    return float(re.findall(r'Gen: ([\d\.]+) MB', output)[0])
        
def get_reclaimed_size(output):
    match = re.findall(r'Gen: ([\d\.]+) MB \(([\d\.]+)%\)', output)
    (reclaimed_size, reclaimed_pct) = map(float, match[0])
    return reclaimed_size, reclaimed_pct

def check_reclaimed(output):
    # basic checks
    to_match = [r'Starting experiment', 'Finishing experiment', r"['x1', 'x2']", r'Gen RAM Free']
    for s in to_match: assert re.search(s, output), f"expecting string: {s}"

    consumed_size = get_consumed_size(output)
    (reclaimed_size, reclaimed_pct) = get_reclaimed_size(output)
    
    # compare: numbers are within 2% equal
    assert isclose(consumed_size, reclaimed_size, rel_tol=0.02), f"Reclaimed all memory: {consumed_size} == {reclaimed_size}"
    assert reclaimed_pct > 99.9, "99.9+% reclaimed"

def check_stats(output, consumed, reclaimed, available):
    consumed_size = get_consumed_size(output)
    (reclaimed_size, reclaimed_pct) = get_reclaimed_size(output)

    # compare with stats
    final_consumed_size_stats  = final_consumed['gen_ram']/2**20
    final_reclaimed_size_stats = final_consumed['gen_ram']/2**20
    # numbers are within 2% equal
    assert isclose(final_consumed_size_stats, consumed_size, rel_tol=0.02), f"Consumed {final_consumed_size_stats} vs reclaimed {reclaimed_size} memory"
    assert isclose(final_reclaimed_size_stats, reclaimed_size, rel_tol=0.02), f"Reclaimed {final_reclaimed_size_stats} vs reclaimed {reclaimed_size} memory"

In [5]:
"""test_version"""
assert ipyexperiments.__version__, "version check"

'test_version'

## Basic container test


In [6]:
%%capture output
exp1 = IPyExperiments(backend='cpu') # consume some cpu ram

x1 = consume_cpu(2**12)
x2 = consume_cpu(2**12)
check_defined(['x1', 'x2'], locals())

del exp1 # finish experiment
check_undefined(['x1', 'x2'], locals())

In [7]:
output = str(output)
print(output)
check_reclaimed(output)

*** Loading backend: cpu

*** Starting experiment...

*** Current state:
Gen RAM Free 15.4 GB | Proc size 67.9 MB



*** Finishing experiment...

*** Deleting the following local variables:
['x1', 'x2']

*** RAM consumed during the experiment:
Gen: 255.8 MB

*** RAM reclaimed at the end of the experiment:
Gen: 255.8 MB (99.99%)

*** Elapsed wallclock time:
00:00:00

*** Current state:
Gen RAM Free 15.4 GB | Proc size 67.9 MB





## Test stats data and selective vars preservation

In [8]:
%%capture output
exp2 = IPyExperiments('cpu') 
x1 = consume_cpu(2**12)

consumed, reclaimed, available = exp2.get_stats()
assert consumed
assert reclaimed
assert available
#print(consumed, reclaimed, available)

exp2.keep_var_names('consumed', 'reclaimed', 'available')

x2 = consume_cpu(2**12) 

consumed, reclaimed, available = exp2.get_stats()
print(consumed, reclaimed, available)

check_defined(['x1', 'x2', 'consumed', 'reclaimed', 'available'], locals())
    
final_consumed, final_reclaimed, final_available = exp2.finish() # finish experiment
print("\nNumerical data:\n", final_consumed, final_reclaimed, final_available)

check_defined(['consumed', 'reclaimed', 'available'], locals())
check_undefined(['x1', 'x2'], locals())

In [9]:
output = str(output)
print(output)
check_reclaimed(output)
check_stats(output, consumed, reclaimed, available)

*** Loading backend: cpu

*** Starting experiment...

*** Current state:
Gen RAM Free 15.4 GB | Proc size 67.9 MB


{'gen_ram': 268435456, 'gpu_ram': 0} {'gen_ram': 0, 'gpu_ram': 0} {'gen_ram': 16312975360, 'gpu_ram': 0}

*** Finishing experiment...

*** Deleting the following local variables:
['exp2', 'x1', 'x2']

*** Keeping the following local variables:
['available', 'consumed', 'reclaimed']

*** RAM consumed during the experiment:
Gen: 256.0 MB

*** RAM reclaimed at the end of the experiment:
Gen: 255.9 MB (99.98%)

*** Elapsed wallclock time:
00:00:00

*** Current state:
Gen RAM Free 15.4 GB | Proc size 67.9 MB



Numerical data:
 {'gen_ram': 268435456, 'gpu_ram': 0} {'gen_ram': 268369920, 'gpu_ram': 0} {'gen_ram': 16581181440, 'gpu_ram': 0}



## Using the context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [10]:
%%capture output
with IPyExperiments('cpu'): 
    x1 = consume_cpu(2**12)
    x2 = consume_cpu(2**12)
    
check_undefined(['x1', 'x2'], locals())

In [11]:
output = str(output)
print(output)
check_reclaimed(output)

*** Loading backend: cpu

*** Starting experiment...

*** Current state:
Gen RAM Free 15.4 GB | Proc size 67.9 MB



*** Finishing experiment...

*** Deleting the following local variables:
['x1', 'x2']

*** RAM consumed during the experiment:
Gen: 256.0 MB

*** RAM reclaimed at the end of the experiment:
Gen: 256.0 MB (100.00%)

*** Elapsed wallclock time:
00:00:00

*** Current state:
Gen RAM Free 15.4 GB | Proc size 68.0 MB





In [12]:
%%capture output
with IPyExperiments('cpu') as exp: 
    x1 = consume_cpu(2**12)
    z = "some data"
    x2 = consume_cpu(2**12)
    exp.keep_var_names('z')
print(z)
check_defined(['z'], locals())
check_undefined(['x1', 'x2'], locals())

In [13]:
output = str(output)
print(output)
check_reclaimed(output)

*** Loading backend: cpu

*** Starting experiment...

*** Current state:
Gen RAM Free 15.4 GB | Proc size 68.0 MB



*** Finishing experiment...

*** Deleting the following local variables:
['exp', 'x1', 'x2']

*** Keeping the following local variables:
['z']

*** RAM consumed during the experiment:
Gen: 256.0 MB

*** RAM reclaimed at the end of the experiment:
Gen: 256.0 MB (100.00%)

*** Elapsed wallclock time:
00:00:00

*** Current state:
Gen RAM Free 15.4 GB | Proc size 68.0 MB


some data



## Deleting previously defined variables

Here we test that we **cannot** correctly detect and delete variables used in the scope of the experiment, but which were already defined prior to the experiment.

In [14]:
%%capture output
x1 = 10
x2 = 20
with IPyExperiments('cpu'): 
    x1 = 10
    x2 = 10000
    x3 = consume_cpu(2**12)
# x1 is the same value, so we can't tell the difference whether it was created before the experiment or not
# x2 is different from previous value, but it could have been modified by some function indirectly and not used directly in the experiment
check_defined(['x1', 'x2'], locals())
# x3 is newly defined, so it gets deleted
check_undefined(['x3'], locals())

In [15]:
output = str(output)
print(output)
check_reclaimed(output)

*** Loading backend: cpu

*** Starting experiment...

*** Current state:
Gen RAM Free 15.4 GB | Proc size 68.0 MB



*** Finishing experiment...

*** Deleting the following local variables:
['x3']

*** RAM consumed during the experiment:
Gen: 127.9 MB

*** RAM reclaimed at the end of the experiment:
Gen: 127.9 MB (100.00%)

*** Elapsed wallclock time:
00:00:00

*** Current state:
Gen RAM Free 15.4 GB | Proc size 68.0 MB





In [16]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>